### Step 1: Setting Up MySQL Connection
First, we'll set up a connection to the MySQL database using the `mysql-connector-python` library.



### Step 2: Connecting to MySQL Database
We'll create a function to establish a connection to the MySQL server. Ensure that your MySQL server is running and the credentials provided are correct.



In [2]:
import os
from dotenv import load_dotenv
import mysql.connector
from mysql.connector import Error

# Load environment variables from .env file
load_dotenv()

def create_connection():
    """
    Establishes a connection to the MySQL database.

    Returns:
        connection (mysql.connector.connection_cext.CMySQLConnection): MySQL connection object
    """
    try:
        connection = mysql.connector.connect(
            host=os.getenv('MYSQL_HOST'),
            user=os.getenv('MYSQL_USER'),
            password=os.getenv('MYSQL_PASSWORD'),
            port=int(os.getenv('MYSQL_PORT'))  # Default MySQL port
        )
        if connection.is_connected():
            print("Connection to MySQL established successfully.")
            return connection
    except Error as e:
        print(f"Error connecting to MySQL: {e}")
        return None


# Testing the connection
conn = create_connection()


Connection to MySQL established successfully.


### Step 3: Creating the Database and Table
We'll create the `user_management` database and the `users` table with the specified fields. The `email` field is set to be unique to prevent duplicate entries.



In [3]:

def create_database_and_table(connection):
    """
    Creates the 'user_management' database and 'users' table if they do not exist.

    Args:
        connection (mysql.connector.connection_cext.CMySQLConnection): MySQL connection object
    """
    try:
        cursor = connection.cursor()

        # Create the database if it doesn't exist
        cursor.execute("CREATE DATABASE IF NOT EXISTS user_management")
        print("Database 'user_management' is ready.")

        # Select the created database
        connection.database = 'user_management'

        # Create the users table
        create_table_query = """
        CREATE TABLE IF NOT EXISTS users (
            id INT AUTO_INCREMENT PRIMARY KEY,
            name VARCHAR(100) NOT NULL,
            email VARCHAR(100) NOT NULL UNIQUE,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
        """
        cursor.execute(create_table_query)
        print("Table 'users' is ready.")

        cursor.close()
    except Error as e:
        print(f"Error creating database or table: {e}")


# Creating the database and table
if conn:
    create_database_and_table(conn)


Database 'user_management' is ready.
Table 'users' is ready.


### Step 4: Implementing CRUD Operations
We'll implement the following CRUD operations for the `users` table:
1. **Create a New User**
2. **Read User Details by ID**
3. **Update User's Email**
4. **Delete a User by ID**

Each function includes error handling and informative print statements for better clarity.

#### 4.1 Create a New User



#### 4.1 Create a New User
This function inserts a new user into the `users` table. It ensures that the `email` is unique and handles any exceptions that may occur during the insertion.



In [19]:
# Part 4.1: Create User
def create_user(connection, username, email, password):
    """Inserts a new user into the users table."""
    try:
        cursor = connection.cursor()
        insert_query = """
        INSERT INTO users (username, email, password)
        VALUES (%s, %s, %s)
        """
        cursor.execute(insert_query, (username, email, password))
        connection.commit()
        print("User created successfully")
    except Error as e:
        print(f"Error creating user: {e}")

# Creating a user with the correct column names
create_user(conn, "rohith", "rohith@abc.com", "1234")

Error creating user: 1062 (23000): Duplicate entry 'rohith' for key 'users.username'


#### 4.2 Read User Details by ID
This function retrieves user details based on the provided `id`. It fetches the user data and displays it in a readable format.



In [20]:
def read_user_by_id(connection, user_id):
    """
    Retrieves user details by ID.

    Args:
        connection (mysql.connector.connection_cext.CMySQLConnection): MySQL connection object
        user_id (int): ID of the user to retrieve

    Returns:
        None
    """
    try:
        cursor = connection.cursor(dictionary=True)
        select_query = "SELECT * FROM users WHERE id = %s"
        cursor.execute(select_query, (user_id,))
        user = cursor.fetchone()
        if user:
            print("User Details:")
            print(f"ID: {user['id']}")
            print(f"Username: {user['username']}")
            print(f"Email: {user['email']}")
        else:
            print(f"No user found with ID {user_id}.")
        cursor.close()
    except Error as e:
        print(f"Error reading user: {e}")

#### 4.3 Update User's Email
This function updates the email address of a user identified by their `id`. It ensures that the new email is unique.



In [21]:
def update_user_email(connection, user_id, new_email):
    """
    Updates the email of a user based on their ID.

    Args:
        connection (mysql.connector.connection_cext.CMySQLConnection): MySQL connection object
        user_id (int): ID of the user to update
        new_email (str): New email address

    Returns:
        None
    """
    try:
        cursor = connection.cursor()
        update_query = """
        UPDATE users
        SET email = %s
        WHERE id = %s
        """
        cursor.execute(update_query, (new_email, user_id))
        connection.commit()
        if cursor.rowcount:
            print(f"User ID {user_id} email updated to '{new_email}'.")
        else:
            print(f"No user found with ID {user_id}.")
        cursor.close()
    except Error as e:
        print(f"Error updating user: {e}")

#### 4.4 Delete a User by ID
This function deletes a user from the `users` table based on their `id`. It confirms the deletion by checking the affected rows.



In [22]:
def delete_user_by_id(connection, user_id):
    """
    Deletes a user from the users table based on their ID.

    Args:
        connection (mysql.connector.connection_cext.CMySQLConnection): MySQL connection object
        user_id (int): ID of the user to delete

    Returns:
        None
    """
    try:
        cursor = connection.cursor()
        delete_query = "DELETE FROM users WHERE id = %s"
        cursor.execute(delete_query, (user_id,))
        connection.commit()
        if cursor.rowcount:
            print(f"User ID {user_id} deleted successfully.")
        else:
            print(f"No user found with ID {user_id}.")
        cursor.close()
    except Error as e:
        print(f"Error deleting user: {e}")

### Step 5: Testing the Implementation
We'll define test cases to validate each CRUD operation. These tests will demonstrate the functionality and handle potential edge cases.



#### 5.1 Test Case: Create a New User

In [25]:
print("\n--- Test Case 1: Create a New User ---")
create_user(conn, "tannu", "tannu@example.com", "12334")




--- Test Case 1: Create a New User ---
Error creating user: 1062 (23000): Duplicate entry 'tannu' for key 'users.username'


#### 5.2 Test Case: Read User Details by ID



In [27]:

print("\n--- Test Case 2: Read User Details by ID ---")
read_user_by_id(conn, 2)  # Assuming the first user has ID 1



--- Test Case 2: Read User Details by ID ---
User Details:
ID: 2
Username: rohith
Email: rohith@abc.com


#### 5.3 Test Case: Update User's Email

In [28]:
print("\n--- Test Case 3: Update User's Email ---")
update_user_email(conn, 3, "Tannu@example.com")


--- Test Case 3: Update User's Email ---
User ID 3 email updated to 'Tannu@example.com'.


#### 5.4 Test Case: Delete a User by ID

In [29]:
print("\n--- Test Case 4: Delete a User by ID ---")
delete_user_by_id(conn, 7) 


--- Test Case 4: Delete a User by ID ---
User ID 7 deleted successfully.


#### 5.5 Test Case: Attempt to Read Deleted User

In [30]:
print("\n--- Test Case 5: Attempt to Read Deleted User ---")
read_user_by_id(conn, 7) 


--- Test Case 5: Attempt to Read Deleted User ---
No user found with ID 7.


### Step 6: Closing the Connection

In [32]:
def close_connection(connection):
    """Closes the database connection."""
    if connection.is_connected():
        connection.close()
        print("Connection closed")

# Close the connection
close_connection(conn)

Connection closed


### Step 7: Considerations
#### 7.1 Strengths and Weaknesses of MySQL

**Strengths:**
- **Reliability and Stability:** MySQL is known for its robustness and has been widely adopted in various industries.
- **Ease of Use:** It offers a straightforward setup and is user-friendly, making it accessible for beginners and professionals alike.
- **Strong Community Support:** A large community ensures ample resources, tutorials, and third-party tools.
- **ACID Compliance:** Ensures reliable transactions with atomicity, consistency, isolation, and durability.
- **Performance:** Optimized for read-heavy operations, making it suitable for applications with frequent data retrieval.

**Weaknesses:**
- **Scalability Limitations:** While MySQL can handle large databases, scaling horizontally (sharding) can be complex compared to some NoSQL databases.
- **Rigid Schema:** Changes to the database schema can be cumbersome and may require significant downtime.
- **Limited Support for Complex Data Types:** Not as flexible as NoSQL databases in handling unstructured or semi-structured data.

#### 7.2 Scalability Beyond 10TB of Data

Handling massive datasets requires strategic planning to ensure performance and reliability. Here are techniques to scale MySQL beyond 10TB:

**1. Partitioning:**
   - **Definition:** Dividing a large table into smaller, more manageable pieces called partitions.
   - **Benefits:** Improves query performance and makes maintenance tasks like backups and archiving more efficient.
   - **Types:**
     - **Range Partitioning:** Divides data based on ranges of values (e.g., dates).
     - **List Partitioning:** Based on predefined lists of values.
     - **Hash Partitioning:** Distributes data based on a hashing function, ensuring an even distribution.

**2. Sharding:**
   - **Definition:** Splitting the database horizontally by distributing data across multiple servers or instances.
   - **Benefits:** Allows the database to handle more data and higher throughput by leveraging multiple machines.
   - **Implementation:**
     - **Application-Level Sharding:** The application determines which shard to query based on a sharding key.
     - **Proxy-Based Sharding:** A proxy layer manages the distribution of queries to the appropriate shards.

**3. Replication:**
   - **Definition:** Creating copies of the database across different servers.
   - **Benefits:** Enhances data availability and load balancing by directing read operations to replicas.
   - **Types:**
     - **Master-Slave Replication:** One master handles write operations, and multiple slaves handle read operations.
     - **Master-Master Replication:** Multiple masters handle both read and write operations, providing redundancy.

**4. Storage Optimization:**
   - **Use of Solid-State Drives (SSDs):** Faster read/write operations compared to traditional HDDs.
   - **Data Compression:** Reduces storage requirements and can improve I/O performance.

**5. Index Optimization:**
   - **Proper Indexing:** Ensures that queries run efficiently, reducing the load on the database.
   - **Regular Maintenance:** Rebuilding and optimizing indexes to maintain performance.

**6. Caching:**
   - **Implementing Caching Layers:** Tools like Redis or Memcached can store frequently accessed data in memory, reducing the load on MySQL.

**7. Monitoring and Maintenance:**
   - **Regular Monitoring:** Tools like MySQL Enterprise Monitor or third-party solutions help track performance metrics.
   - **Proactive Maintenance:** Addressing potential issues before they escalate, ensuring the database remains healthy.

By implementing these strategies, MySQL can effectively handle datasets that exceed 10TB, ensuring scalability, performance, and reliability.



### Conclusion
This notebook provides a comprehensive guide to setting up a User Management System using MySQL. It covers establishing a connection, creating the necessary database and table, implementing CRUD operations with proper error handling, and testing each operation. Additionally, it discusses the strengths and weaknesses of MySQL and offers strategies to scale the database beyond 10TB of data. 

Feel free to modify and expand upon this foundation to suit your specific project needs. If you have any further questions or need additional assistance, don't hesitate to ask!
